In [1]:
import psycopg2
from os import getenv
import pandas as pd
from dotenv import load_dotenv

In [91]:
load_dotenv()  # Load variables from .env file

### Connect to ElephantSQL-hosted PostgreSQL
connection = psycopg2.connect(dbname=getenv('PG_DB'), 
                        user=getenv('PG_USER'),
                        password=getenv('PG_PW'), 
                        host=getenv('PG_HOST'))

cursor = connection.cursor()

In [70]:
def q(query):
    try:
        cursor.execute(query)
        print(cursor.fetchall())
    except psycopg2.Error as e:
        # Rollback the transaction
        connection.rollback()
        print("Error:", e)    

In [9]:
q('select * from titanic limit 1')

[(1, '0', '3', 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)]

In [14]:
# view schema
q('''
SELECT column_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name = 'titanic';
''')

[('id', 'integer', None),
 ('survived', 'USER-DEFINED', None),
 ('pclass', 'USER-DEFINED', None),
 ('name', 'character varying', 200),
 ('sex', 'USER-DEFINED', None),
 ('age', 'double precision', None),
 ('sib_spouse', 'integer', None),
 ('parents_children', 'integer', None),
 ('fare', 'double precision', None)]

In [92]:
# How many passengers survived, and how many died?
q('''
SELECT survived, COUNT(*)
FROM titanic
GROUP BY survived
''')

[('1', 342), ('0', 545)]


In [93]:
# How many passengers were in each class?
q('''
SELECT pclass, COUNT(*)
FROM titanic
GROUP BY pclass
''')

[('3', 487), ('2', 184), ('1', 216)]


In [96]:
# How many passengers survived/died within each class?
q('''
SELECT pclass, COUNT(*)
FROM titanic
WHERE survived = '1'
GROUP BY pclass
''')

[('3', 119), ('2', 87), ('1', 136)]


In [97]:
# What was the average age of survivors vs nonsurvivors?
q('''
SELECT survived, AVG(age)
FROM titanic
GROUP BY survived
''')

[('1', 28.4083918128655), ('0', 30.1385321100917)]


In [98]:
# What was the average age of each passenger class?
q('''
SELECT pclass, AVG(age)
FROM titanic
GROUP BY pclass
''')

[('3', 25.1887474332649), ('2', 29.8686413043478), ('1', 38.7889814814815)]


In [99]:
# What was the average fare by passenger class? 
q('''
SELECT pclass, AVG(fare)
FROM titanic
GROUP BY pclass
''')

[('3', 13.7077073921971), ('2', 20.6621831521739), ('1', 84.1546874999999)]


In [100]:
# What was the average fare by survival? 
q('''
SELECT survived, AVG(fare)
FROM titanic
GROUP BY survived
''')

[('1', 48.3954076023392), ('0', 22.2085840366972)]


In [101]:
# How many siblings/spouses aboard on average, by passenger class?
q('''
SELECT pclass, AVG(sib_spouse)
FROM titanic
GROUP BY pclass
''')

[('3', Decimal('0.62012320328542094456')), ('2', Decimal('0.40217391304347826087')), ('1', Decimal('0.41666666666666666667'))]


In [105]:
# Do any passengers have the same name?
q('''
SELECT name, COUNT(*) 
FROM titanic 
GROUP BY name 
HAVING COUNT(*) > 1 
''')

# # alternative approach
# q('''
# SELECT COUNT(DISTINCT name) 
# FROM titanic  
# ''')


[]
